# Lab 4


## Phạm Đăng Đan
## MSSV: 19529651

# Câu 1 (3 điểm): 

Cho một RDD trong Spark chứa danh sách lớp như cell dưới đây

(a)	(1 điểm) Sử dụng Pyspark, liệt kê các tên trong danh sách trên (mỗi tên chỉ xuất hiện một lần).

(b)	(2 điểm) Sử dụng hàm map() và reduceByKey() (Pyspark), tìm số sinh viên tương ứng với mỗi tên trong danh sách từ Câu (1a).


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lab4_1").getOrCreate()
sc = spark.sparkContext

list_class = ['Tuan', 'Hoang', 'Minh', 'Nga', 'Tam', 'Tuan', 'Minh', 'Tam', 'Tuan', 'Phuong']
rdd_class = sc.parallelize(list_class)                                                  

In [10]:
# print(rdd_class)
# câu a
# Sử dụng hàm distinct() để lấy các tên duy nhất từ RDD
unique_names = rdd_class.distinct()

# Thu thập kết quả về một danh sách Python và in ra
unique_names_list = unique_names.collect()
print(unique_names_list)

['Nga', 'Tuan', 'Tam', 'Minh', 'Hoang', 'Phuong']


In [11]:
# câu b
# Đầu tiên, sử dụng hàm map() để tạo cặp (tên, 1) cho mỗi tên trong RDD
name_count_pairs = rdd_class.map(lambda name: (name, 1))

# Tiếp theo, sử dụng hàm reduceByKey() để tính tổng số lần xuất hiện của mỗi tên
name_counts = name_count_pairs.reduceByKey(lambda a, b: a + b)

# Thu thập kết quả về một danh sách Python và in ra
name_counts_list = name_counts.collect()
print(name_counts_list)

[('Nga', 1), ('Tuan', 3), ('Tam', 2), ('Minh', 2), ('Hoang', 1), ('Phuong', 1)]


## Câu 2 (7 điểm)

Trong câu này, bạn sẽ upload file data shakespeare.txt từ thư mục LMS và đếm từ trong file này.

### (2a) (1 điểm) 

Upload file lên LMS và gán vào RDD có tên là **rdd_shp**

In [12]:
import os.path

# Ví dụ
fileName = "./LMS/shakespeare.txt"

if os.path.exists(fileName) == False:
  print("File does not exist")
else:
  print("File exist")

File exist


In [13]:
from pyspark import SparkContext
spark = SparkSession.builder.appName("Lab4_2").getOrCreate()
sc = spark.sparkContext

rdd_shp = sc.textFile(fileName)

assert rdd_shp.take(3)==['The Complete Works of William Shakespeare', '', 'by William Shakespeare'], "Assertion Error"

### (2b) (1 điểm)

Viết hàm **removePunctuation** để chuyển một đoạn văn bản thành chữ thường (lower case) và xoá tất cả các dấu chấm câu. Áp dụng hàm này cho **rdd_shp** để nhận được RDD **rdd_shp_2**

In [14]:
import re
from pyspark import SparkContext

def removePunctuation(text):
    # Sử dụng biểu thức chính quy để loại bỏ dấu chấm câu và chuyển thành chữ thường
    cleaned_text = re.sub(r'[^\w\s]', '', text).lower()
    return cleaned_text

# Tạo SparkContext
sc = spark.sparkContext

In [15]:
# Áp dụng hàm removePunctuation cho RDD rdd_shp để nhận được rdd_shp_2
rdd_shp_2 = rdd_shp.map(removePunctuation)

# Hiển thị một số dòng đầu tiên trong RDD rdd_shp_2 để kiểm tra
for line in rdd_shp_2.take(15):
    print(line)


the complete works of william shakespeare

by william shakespeare




                    contents

    the sonnets
    alls well that ends well
    the tragedy of antony and cleopatra
    as you like it
    the comedy of errors
    the tragedy of coriolanus


### (2c) 2 điểm

Tạo RDD **rdd_words** gồm các words từ **rdd_shp_2** bằng cách split **rdd_shp_2** sử dụng dấu cách (space). In ra 5 từ đầu tiên của **rdd_words** và số lượng từ trong **rdd_words**.

In [16]:
# Tách các từ từ RDD rdd_shp_2 bằng dấu cách và tạo RDD rdd_words
rdd_words = rdd_shp_2.flatMap(lambda line: line.split(" "))

# In ra 5 từ đầu tiên của rdd_words
first_5_words = rdd_words.take(5)
print("5 từ đầu tiên trong rdd_words:", first_5_words)

# Đếm số lượng từ trong rdd_words
word_count = rdd_words.count()
print("Số lượng từ trong rdd_words:", word_count)


5 từ đầu tiên trong rdd_words: ['the', 'complete', 'works', 'of', 'william']
Số lượng từ trong rdd_words: 1222431


### (2d) 1 điểm

Tạo RDD **rdd_words_2** từ **rdd_words** bằng cách xoá tất cả các empty word (''). In ra số lượng từ trong **rdd_words_2**.

In [17]:
# Loại bỏ tất cả các empty word ('') từ RDD rdd_words và tạo RDD rdd_words_2
rdd_words_2 = rdd_words.filter(lambda word: word != '')

# In ra số lượng từ trong rdd_words_2
word_count_2 = rdd_words_2.count()
print("Số lượng từ trong rdd_words_2:", word_count_2)


Số lượng từ trong rdd_words_2: 958702


### (2e) 2 điểm

Sử dụng code từ câu (1b), viết một function nhận một RDD với một danh sách word đầu vào và đếm số lần xuất hiện của mỗi từ. Sử dụng function này để đếm từ trong **rdd_words_2**. **Bravo! Bạn đã hoàn thành việc đếm số lần xuất hiện của tất cả các từ trong toàn bộ các tác phẩm của Shakespeare!** In ra 10 từ được sử dụng nhiều nhất trong toàn bộ các tác phẩm của Shakespeare.

In [18]:
def wordCount(wordListRDD):
    # Sử dụng hàm map() để tạo cặp (từ, 1) cho mỗi từ trong RDD
    word_count_pairs = wordListRDD.map(lambda word: (word, 1))
    
    # Sử dụng hàm reduceByKey() để tính tổng số lần xuất hiện của mỗi từ
    word_counts = word_count_pairs.reduceByKey(lambda a, b: a + b)
    
    # Sử dụng hàm sortBy() để sắp xếp theo số lần xuất hiện giảm dần và lấy 10 từ đầu tiên
    top_10_words = word_counts.sortBy(lambda x: x[1], ascending=False).take(10)
    
    return top_10_words

In [19]:
# Sử dụng hàm countWords để đếm từ trong rdd_words_2
top_10_words_shakespeare = wordCount(rdd_words_2)

# In ra 10 từ được sử dụng nhiều nhất
print("10 từ được sử dụng nhiều nhất trong toàn bộ các tác phẩm của Shakespeare:")
for word, count in top_10_words_shakespeare:
    print(f"{word}: {count} lần")

10 từ được sử dụng nhiều nhất trong toàn bộ các tác phẩm của Shakespeare:
the: 30022 lần
and: 28328 lần
i: 21963 lần
to: 20684 lần
of: 18689 lần
a: 16131 lần
you: 14363 lần
my: 13182 lần
in: 12198 lần
that: 11762 lần
